## 라이브러리

In [ ]:
import re
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__)))))

import torch
import pandas as pd
from transformers import AutoModel, AutoTokenizer
from google.cloud import storage

## 클래스

In [ ]:
class KoSimCSE:
    """
    KoSimCSE 텍스트 유사도 모델 활용
    텍스트 Embedding 전환
    """


    def __init__(self) -> None:
        self.MODEL_NAME='BM-K/KoSimCSE-roberta-multitask' # ==> 모델 이름 정의
        self.BUCKET_NAME='law-search'  # ==> GCS 저장소 bucket name


    def convert_text_to_embedding(self, sub_df: pd.DataFrame) -> pd.DataFrame:
        """
        Text 데이터 embedding 데이터 변환
        """
        # 컬럼 변수
        col_embedding='embedding'

        # 모델 & 토크나이저
        model=AutoModel.from_pretrained(self.MODEL_NAME)
        tokenizer=AutoTokenizer.from_pretrained(self.MODEL_NAME)

        #Mean Pooling - Take attention mask into account for correct averaging
        def mean_pooling(model_output, attention_mask):
            token_embeddings = model_output[0] #First element of model_output contains all token embeddings
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

        embedding_list=[]
        for _, row in sub_df.iterrows():
            law_text=row.law
            
            # 전처리 pattern 및 전처리 진행
            pattern=r'\<(.*?)\>'
            prepro_text=re.sub(pattern, '', law_text).strip()

            # 특수기호 제거 및 띄어쓰기 반복 변형
            prepro_text=re.sub(r' +', ' ',re.sub(r'[^가-힣a-zA-Z0-9]', ' ', prepro_text)).strip()

            # Tokenize sentences
            encoded_input = tokenizer(prepro_text, padding=True, truncation=True, return_tensors='pt')

            # Compute token embeddings
            with torch.no_grad():
                model_output = model(**encoded_input)
            
            # Perform pooling. In this case, mean pooling.
            sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
            # 데이터 정립
            embedding_list.append(sentence_embeddings[0].tolist())

        prepro_sub_df=sub_df.copy()
        prepro_sub_df[col_embedding]=embedding_list
        
        return prepro_sub_df

    
    def local_to_gcs(self, file_name):
        """
        로컬에 저장된 파일 GCS bucket에 업로드
        """
        storage_client = storage.Client()
        bucket=storage_client.bucket(self.BUCKET_NAME)
        local_path=os.path.join(os.path.abspath(os.getcwd()), file_name)  # local에서 업로드할 파일 경로
        gcs_upload_path= "law/"+file_name   # 업로드 할 GCS 경로
        blob=bucket.blob(gcs_upload_path)   # 업로드 할 bucket 세팅
        blob.upload_from_filename(local_path)   # 업로드
        print(f'[INFO] Local to GCS bucket Upload completely -> file : {local_path}')

## Test

In [ ]:
kosim_cls=KoSimCSE()
test_filnm = "./test.csv"
test_df = pd.read_csv(test_filnm)
prv_embedding_df=kosim_cls.convert_text_to_embedding(test_df)    # embedding 데이터 만들기
prv_file_name='test_embedding.csv'
prv_embedding_df.to_csv(prv_file_name, index=False, encoding='utf-8')   # embedding 데이터 저장
kosim_cls.local_to_gcs(prv_file_name)   # embedding 데이터 업로드
os.remove(prv_file_name)    # remove csv file from local
print(f"[INFO] Remove {prv_file_name} file completely")